# Shape-E Text-to-3D, Image-to-3D, Generate mesh

Generate 3D assets with a text prompt or an image.

In [ ]:
# Install required libraries
!pip3 install -q diffusers transformers accelerate trimesh gradio

In [ ]:
# Import modules
# import spaces
import gradio as gr
import numpy as np
import PIL.image
import tempfile
import torch
import trimesh
from diffusers import ShapEPipeline, ShapEImg2ImgPipeline
from diffusers.utils import export_to_ply

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(f"device: {device}")

In [ ]:
# Build text-to-3D pipeline using Diffusers library
pipe_text3d = ShapEPipeline.from_pretrained(
    "openai/shap-e",
    torch_dtype=torch.float16,
    variant="fp16"
)

In [ ]:
# Build image-to-3D pipeline using Diffusers library
pipe_image3d = ShapEImg2ImgPipeline.from_pretrained(
    "openai/shap-e",
    torch_dtype=torch.float16,
    variant="fp16"
)

In [ ]:
pipe_text3d = pipe_text3d.to(device)
pipe_image3d = pipe_image3d.to(device)

In [ ]:
# Define function to convert ply file to glb file
def ply_to_glb(ply_path):
    mesh = trimesh.load(ply_path)
    rot = trimesh.transformations.rotation_matrix(-np.pi / 2, [1, 0, 0])
    mesh = mesh.apply_transform(rot)
    # rot = trimesh.transformations.rotation_matrix(np.pi, [0, 1, 0])
    # mesh = mesh.apply_transform(rot)
    mesh_path = tempfile.NamedTemporaryFile(suffix=".glb", delete=False)
    mesh.export(mesh_path.name, file_type="glb")
    
    return mesh_path.name

In [ ]:
# Define function to generate 3D assets with text prompt
def text_to_3d(prompt, guidance_scale=15.0, num_steps=64):
    """
    Generate 3D assets with a text prompt.
    """
    images = pipe_text3d(
        prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=num_steps,
        # frame_size=256,
        output_type="mesh"
    ).images
    
    ply_path = tempfile.NamedTemporaryFile(suffix=".ply", delete=False, mode="w+b")
    export_to_ply(images[0], ply_path.name)
    
    return to_glb(ply_path.name)

In [ ]:
# Define function to generate 3D assets with image
def image_to_3d(image, guidance_scale=3.0, num_steps=64):
    """
    Generates 3D assets with an image.
    """
    images = pipe_image3d(
        image, 
        guidance_scale=guidance_scale,
        num_inference_steps=num_steps,
        # frame_size=256,
        output_type="mesh"
    ).images
    
    ply_path = tempfile.NamedTemporaryFile(suffix=".ply", delete=False, mode="w+b")
    export_to_ply(images[0], ply_path.name)
    
    return to_glb(ply_path.name)

In [ ]:
# Define Gradio application
with gr.Blocks() as demo:
    gr.Markdown("# Shap-E Text-to-3D & Image-to-3D")
    with gr.Tabs():
        with gr.Tab(label="Text-to-3D"):
            with gr.Group():
                with gr.Row():
                    prompt = gr.Text(
                        label="Prompt",
                        show_label=False,
                        max_lines=1,
                        placeholder="Enter prompt",
                        container=False
                    )
                    run_button = gr.Button(
                        label="Run",
                        scale=0
                    )
                result = gr.Model3D(
                    label="Result",
                    show_label=False
                )
            gr.on(
                triggers=[prompt.submit, run_button.click],
                fn=text_to_3d,
                inputs=[
                    prompt#,
                    # guidance_scale,
                    # num_inference_steps
                ],
                outputs=result,
                api_name="text-to-3d",
                concurrency_limit=1,
                concurrency_id="gpu"
            )
        with gr.Tab(label="Image-to-3D"):
            create_demo_image_to_3d(model)

In [ ]:
# Set queue with default settings
demo.queue()

In [ ]:
# Start Gradio application
demo.launch()

In [ ]:
# Close Gradio application
demo.close()